In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import torch
from transformers import BertConfig
from fastformer import FastformerEncoder

In [ ]:
config=BertConfig.from_json_file('fastformer.json')

In [ ]:
model = FastformerEncoder(config)

In [ ]:
batch_size = 8
seq_len = 50 
dummy_input = torch.randn(batch_size, seq_len, config.hidden_size)
dummy_mask = torch.ones(batch_size, seq_len)

In [ ]:
model.eval()
with torch.no_grad():
    output = model(dummy_input, dummy_mask)

In [ ]:
print(dummy_input.shape)
print(output.shape)

In [1]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

c:\Users\madmo\miniconda3\envs\dl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = bert(**encoded_input)
print(output.last_hidden_state[0].shape)

torch.Size([12, 768])


In [ ]:
print(f"Token: {tokenizer.pad_token} | ID: {tokenizer.pad_token_id}")

In [55]:
from data_utils import build_user_id_map, build_news_id_map, build_news_token, build_edge_index, build_behaviors
import pickle 

# build_news_id_map()
# build_user_id_map()
# build_news_token(news_id_map)

In [35]:
import numpy as np
news_token = np.load('data/MINDsmall_train/news_token.npy')

In [ ]:
news_token = torch.from_numpy(news_token)
attention_mask = (news_token != 0).long()

In [36]:
with open('data/MINDsmall_train/user_id_map.pkl', 'rb') as f:
    user_id_map = pickle.load(f)

In [44]:
with open('data/MINDsmall_train/news_id_map.pkl', 'rb') as f:
    news_id_map = pickle.load(f)

In [56]:
data_path = 'data/MINDsmall_train/'
edge_index = build_edge_index(data_path + 'behaviors.tsv', news_id_map=news_id_map)

100%|██████████| 153727/153727 [00:03<00:00, 47888.50it/s]


In [57]:
build_behaviors(data_path + 'behaviors.tsv', user_id_map, news_id_map, 'behaviors.pkl', 50)

156965it [00:03, 51401.92it/s]


In [69]:
with open('behaviors.pkl', 'rb') as f:
    behaviors_data = pickle.load(f)

In [5]:
from data import MINDDataset
from data_utils import collate_fn

In [4]:
mind_data = MINDDataset('behaviors.pkl')

In [15]:
from torch.utils.data import DataLoader

dataloader = DataLoader(mind_data, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [16]:
batch = next(iter(dataloader))

In [17]:
print(batch['user_idx'])
print(batch['history'].shape)
print(batch['candidates'].shape)
print(batch['seed_nodes'])

tensor([15821, 47644, 21144, 37986, 21557, 32458,  7101, 30888, 15513, 48904,
         8743, 45704,  2744, 43586, 36398, 48255,   168,  8199, 12690, 33131,
        33408,  7480, 12129, 18604, 42959, 17320, 10746, 10933,  7288, 30776,
        43841,  2936])
torch.Size([32, 50])
torch.Size([32, 5])
tensor([   48,   129,   142,   300,   306,   314,   315,   332,   340,   413,
          494,   504,   576,   609,   641,   697,   747,   819,   843,   848,
          946,  1001,  1066,  1106,  1235,  1253,  1341,  1460,  1496,  1500,
         1537,  1567,  1571,  1576,  1610,  1772,  2036,  2059,  2075,  2078,
         2131,  2177,  2243,  2316,  2394,  2505,  2567,  2604,  2704,  2774,
         3021,  3028,  3150,  3171,  3241,  3261,  3382,  3440,  3460,  3585,
         3620,  3867,  3892,  3949,  3959,  4018,  4024,  4080,  4168,  4171,
         4242,  4259,  4292,  4320,  4369,  4385,  4459,  4558,  4572,  4576,
         4618,  4672,  4722,  4860,  4891,  4946,  4959,  5089,  5119,  5170,
